In [41]:
import numpy as np
import pandas as pd 

In [42]:
# 고용표 데이터 

# 맥 
data = pd.read_excel('/Users/dongyokim/main/jupyter/2022:Summer/프로젝트/2015-19_고용표_통합소분류_취업자수 및 근로시간.xlsx', sheet_name='취업자수 및 피용자수(상품)', usecols=[1, 2, 3, 4, 5 ,6 ,7, 8, 9, 10, 11, 12], skiprows=[0, 1, 2, 3, 4])

/Users/dongyokim/miniforge3/envs/crawl/lib/python3.9/site-packages/pandas/io/excel/_base.py:1272: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  return self._reader.parse(


In [43]:
data = data.iloc[:166, :]
data.reset_index(drop=True, inplace=True)
    
data.rename(columns={'Unnamed: 1':'상품분류'}, inplace=True)
data.set_index('상품분류', inplace=True)
    
year = int(input('실측치의 연도를 입력해주세요. '))

data.rename(columns={'취업자수':'취업자수_{}'.format(year)}, inplace=True)
data.rename(columns={'피용자수':'피용자수_{}'.format(year)}, inplace=True)
idx = 1
for i in range(year+1, year+5):
    data.rename(columns={'취업자수.{}'.format(idx):'취업자수_{}'.format(i)}, inplace=True)
    data.rename(columns={'피용자수.{}'.format(idx):'피용자수_{}'.format(i)}, inplace=True)
    idx+=1 

실측치의 연도를 입력해주세요. 2015


In [44]:
data

,취업자수_2015,피용자수_2015,취업자수_2016,피용자수_2016,취업자수_2017,피용자수_2017,취업자수_2018,피용자수_2018,취업자수_2019,피용자수_2019
상품분류,,,,,,,,,,
곡물 및 식량작물,335618.0,10256.1,272668.0,8579.1,252586.8,8204.2,349344.4,8718.2,396968.9,7471.1
채소 및 과실,608618.7,47038.4,600700.4,40869.3,629660.6,40322.1,578541.6,36511.5,575807.7,33967.4
기타작물,139423.6,11637.8,148665.6,12458.3,145968.1,11992.6,133690.1,9462.4,132819.6,8611.2
낙농 및 축우,19482.7,3025.6,21381.5,3655.6,22335.3,3978.2,29990.6,3164.6,46080.5,3373.2
기타 축산,53483.1,9769.6,47895.6,8645.9,49819.7,8902.2,52357.6,10842.9,43167.4,11322.5
...,...,...,...,...,...,...,...,...,...,...
사회단체,221188.2,211259.6,214939.6,204520.2,205331.9,194566.0,227116.5,218919.2,237185.2,223101.4
자동차 및 소비용품 수리서비스,301498.4,170044.9,302393.8,165395.4,303398.5,166637.3,314370.4,175758.1,314204.2,179480.8
개인서비스,662728.1,317642.2,639609.6,295973.9,641479.9,289816.3,606752.7,277358.3,612240.4,278365.7


In [45]:
# 분류표 데이터 불러오기 
group_data =  pd.read_excel('/Users/dongyokim/main/jupyter/2022:Summer/프로젝트/2015_부문분류표.xlsx', usecols=[2, 3, 4, 5 ,6 ,7],skiprows=[0], sheet_name='상품분류표')

In [46]:
# 대분류 소분류 매칭 사전 생성 
def disting_small(data, small_name_count=None):
    data = data.iloc[1:]
    data.reset_index(drop=True, inplace=True)
    data.rename(columns={'Unnamed: 3':'소분류명', 'Unnamed: 5':'중분류명', 'Unnamed: 7':'대분류명'}, inplace=True)
    small_name = list(data['소분류명'].dropna(how=any))
    big_name = list(data['대분류명'].dropna(how=any))
    i = int(input('총 몇개의 대분류로 구성되어 있습니까? : '))
    dic = {}
    idx = 0
    yes_no = input('각 대분류의 포함된 소분류의 수를 이미 작성하신 리스트로 입력하겠습니까? \n 아니면, 직접 입력하겠습니까?(yes or no) ')         
            
    for k in range(i):
        if yes_no == 'yes':
            s = small_name_count[k]
        else:
            s = int(input('대분류 {}: {}에 포함된 소분류가 몇 개 입니까? '.format(k+1, big_name[k])))
            print()
        for j in range(idx, idx+s):
            idx += 1
            dic[small_name[j]] = big_name[k]
        print()
    print('완료')
    return dic, big_name


# 재분류 사전 생성 함수(소분류 수준)
def re_disting_small(dic, list_or_count=None, small_name_list=None):
    s = int(input('새로운 분류를 몇 개 추가할 것인가요? '))
    print()
    print()
    yes_or_no = input('이미 튜플 세트로 정리한 분류 이름과 개수 리스트가 있으신가요?(yes or no) ')
    print()
    yes_or_no2 = input('해당 분류에 속할 소분류 이름을 정리한 리스트가 있으신가요?(yes or no) ')        
    
    for i in range(s):
        if yes_or_no == 'yes':
            name = list_or_count[i][0]
            count_d = list_or_count[i][1]
        else:
            print('새로운 분류 이름과 해당 분류에 포함되는 소분류의 개수를 입력해주세요')
            name = input('새로운 분류 이름: ')
            count_d = int(input('{}에 포함될 소분류 개수: '.format(name)))
        print()
        for k in range(count_d):
            if yes_or_no2 == 'yes':
                small_name = small_name_list[i][k]
            else:
                small_name = input('{}에 포함될 {}번 째 소분류명을 입력해주세요 '.format(name, k+1))
            dic[small_name] = name 
        print(name, '분류 생성 완료')
        print()
    print('재분류 완료')
    return dic     

In [47]:
# 각 대분류의 소분류 수 
small_name_count = [8, 4, 11, 6, 6, 3, 13, 5, 7, 4, 9, 6, 11, 7, 2, 1, 3, 4, 7, 1, 10, 2, 8, 4, 
                    3, 5, 4, 2, 1, 2, 2, 3, 1
                   ]

In [48]:
big_dict, big_name = disting_small(group_data, small_name_count)

/Users/dongyokim/miniforge3/envs/crawl/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


총 몇개의 대분류로 구성되어 있습니까? : 33
각 대분류의 포함된 소분류의 수를 이미 작성하신 리스트로 입력하겠습니까? 
 아니면, 직접 입력하겠습니까?(yes or no) yes

































완료


In [49]:
# 새로 추가할 분류 이름과 해당 분류에 포함될 소분류 갯수 리스트 
new_big_list_or_count = [('공간정보 관련 제조업', 3), ('공간정보 관련 도매업', 1), ('공간정보 관련 출판 및 정보서비스업', 4),('공간정보 관련 기술서비스업', 2), ('공간정보 관련 교육서비스업', 1), ('공간정보 관련 협회 및 단체', 1)]

# 새로운 분류에 포함될 소분류 이름 리스트 
new_small_list = [['통신 및 방송장비', '의료 및 측정기기', '인쇄 및 기록매체 복제'], ['도소매 및 상품중개서비스'], ['소프트웨어 개발 공급', '기타 IT서비스', '정보서비스', '신문 및 출판 서비스'], ['건축·토목 관련 서비스', '기타 과학기술 및 전문 서비스'], ['교육서비스'], ['사회단체']]

# 마지막에 정렬할때 사용할 리스트 생성 
new_name = []
for i, count in new_big_list_or_count:
    new_name.append(i)

In [50]:
# 재분류표 생성 
new_big_dict = re_disting_small(big_dict, new_big_list_or_count, new_small_list)

새로운 분류를 몇 개 추가할 것인가요? 6


이미 튜플 세트로 정리한 분류 이름과 개수 리스트가 있으신가요?(yes or no) yes

해당 분류에 속할 소분류 이름을 정리한 리스트가 있으신가요?(yes or no) yes

공간정보 관련 제조업 분류 생성 완료


공간정보 관련 도매업 분류 생성 완료


공간정보 관련 출판 및 정보서비스업 분류 생성 완료


공간정보 관련 기술서비스업 분류 생성 완료


공간정보 관련 교육서비스업 분류 생성 완료


공간정보 관련 협회 및 단체 분류 생성 완료

재분류 완료


In [51]:
 # 재분류 열 추가 생성 함수(대분류 수준)
def dic_pre(x, dic):
    try:
        return dic[x]
    except:
        return 0

In [52]:
# 자동 부분합 함수(대분류 수준)
def groupping(data, dic, disting_standard, metrix_count, new_name=None):
    
    data['대분류'] = list(pd.Series(data.index.values).apply(lambda x: dic_pre(x, dic)))
    data = data.replace(0, np.NaN)

    metrix_a_1 = data.loc[:'계', :].groupby('대분류').sum()
    metrix_b_1 = data.loc['계':, :]

    big_name_list_index = disting_standard[:metrix_count]
    if new_name != None:
        big_name_list_index.extend(new_name)
    big_name_list_index.extend(list(metrix_b_1.index))
    big_name = big_name_list_index.copy()
    metrix_total_1 = pd.concat([metrix_a_1, metrix_b_1], axis=0)

    metrix_total_1.drop('대분류', axis=1, inplace=True)
    data.drop('대분류', axis=1, inplace=True)
    metrix_total_1 = metrix_total_1.reindex(index=big_name_list_index)
    
    # 멀티 인덱스로 열을 전처리하고 싶은 경우 사용 
#     columns = [['2015년','2015년', '2016년','2016년',  '2017년','2017년', '2018년','2018년', '2019년', '2019년'], 
#               ['취업자수', '피용자수', '취업자수', '피용자수', '취업자수', '피용자수', '취업자수', '피용자수', '취업자수', '피용자수']]
    
    
#     df = pd.DataFrame(data=metrix_total_1.values, columns=columns, index=metrix_total_1
#                      .index)
    
    return metrix_total_1, big_name

In [53]:
data_37, big_name = groupping(data, new_big_dict, big_name, 33, new_name)

In [54]:
data_37

,취업자수_2015,피용자수_2015,취업자수_2016,피용자수_2016,취업자수_2017,피용자수_2017,취업자수_2018,피용자수_2018,취업자수_2019,피용자수_2019
농림수산품,1227523.5,112069.4,1163211.9,106527.6,1170876.4,103631.2,1222491.4,105214.3,1274565.3,101980.4
광산품,11960.0,10874.9,17430.3,17185.6,20967.3,20800.7,17541.6,17473.2,15385.7,14338.6
음식료품,308474.2,237796.6,326307.6,249776.1,339589.1,264638.6,337359.9,262279.0,332904.5,264835.6
섬유 및 가죽제품,333025.9,261883.7,306319.2,232928.6,282891.7,214709.8,268723.3,206277.0,241658.1,181273.8
"목재 및 종이, 인쇄",107235.1,93023.0,104377.5,89187.7,110336.1,94324.0,103990.5,90803.6,95307.8,85510.9
석탄 및 석유제품,11766.8,11601.3,10165.2,10152.8,11091.7,11077.3,10808.2,10620.5,11197.5,11197.5
화학제품,411165.0,378061.0,426457.1,390742.2,440977.9,406055.4,449911.4,417124.9,434660.0,400718.1
비금속광물제품,109621.2,96185.0,116672.0,99408.4,112223.2,96609.6,111340.5,96411.3,105522.9,89355.8
1차 금속제품,148047.7,141684.4,143847.0,135803.0,132441.1,124852.0,130703.8,124353.7,125987.7,120328.3
금속가공제품,371578.7,316951.4,373572.3,320574.4,347619.6,291500.1,347063.9,291791.3,339957.5,291792.3


In [55]:
data_37.drop(['도소매 및 상품중개서비스', '교육서비스'], axis=0, inplace=True)

In [56]:
data_37

,취업자수_2015,피용자수_2015,취업자수_2016,피용자수_2016,취업자수_2017,피용자수_2017,취업자수_2018,피용자수_2018,취업자수_2019,피용자수_2019
농림수산품,1227523.5,112069.4,1163211.9,106527.6,1170876.4,103631.2,1222491.4,105214.3,1274565.3,101980.4
광산품,11960.0,10874.9,17430.3,17185.6,20967.3,20800.7,17541.6,17473.2,15385.7,14338.6
음식료품,308474.2,237796.6,326307.6,249776.1,339589.1,264638.6,337359.9,262279.0,332904.5,264835.6
섬유 및 가죽제품,333025.9,261883.7,306319.2,232928.6,282891.7,214709.8,268723.3,206277.0,241658.1,181273.8
"목재 및 종이, 인쇄",107235.1,93023.0,104377.5,89187.7,110336.1,94324.0,103990.5,90803.6,95307.8,85510.9
석탄 및 석유제품,11766.8,11601.3,10165.2,10152.8,11091.7,11077.3,10808.2,10620.5,11197.5,11197.5
화학제품,411165.0,378061.0,426457.1,390742.2,440977.9,406055.4,449911.4,417124.9,434660.0,400718.1
비금속광물제품,109621.2,96185.0,116672.0,99408.4,112223.2,96609.6,111340.5,96411.3,105522.9,89355.8
1차 금속제품,148047.7,141684.4,143847.0,135803.0,132441.1,124852.0,130703.8,124353.7,125987.7,120328.3
금속가공제품,371578.7,316951.4,373572.3,320574.4,347619.6,291500.1,347063.9,291791.3,339957.5,291792.3


In [57]:
big_name

['농림수산품',
 '광산품',
 '음식료품',
 '섬유 및 가죽제품',
 '목재 및 종이, 인쇄',
 '석탄 및 석유제품',
 '화학제품',
 '비금속광물제품',
 '1차 금속제품',
 '금속가공제품',
 '컴퓨터, 전자 및 광학기기',
 '전기장비',
 '기계 및 장비',
 '운송장비',
 '기타 제조업 제품',
 '제조임가공 및 산업용 장비 수리',
 '전력, 가스 및 증기',
 '수도, 폐기물처리 및 재활용서비스',
 '건설',
 '도소매 및 상품중개서비스',
 '운송서비스',
 '음식점 및 숙박서비스',
 '정보통신 및 방송 서비스',
 '금융 및 보험 서비스',
 '부동산서비스',
 '전문, 과학 및 기술 서비스',
 '사업지원서비스',
 '공공행정, 국방 및 사회보장',
 '교육서비스',
 '보건 및 사회복지 서비스',
 '예술, 스포츠 및 여가 관련 서비스',
 '기타 서비스',
 '기타',
 '공간정보 관련 제조업',
 '공간정보 관련 도매업',
 '공간정보 관련 출판 및 정보서비스업',
 '공간정보 관련 기술서비스업',
 '공간정보 관련 교육서비스업',
 '공간정보 관련 협회 및 단체',
 '계']

In [58]:
big_name.remove('도소매 및 상품중개서비스')
big_name.remove('교육서비스')
big_name.remove('계')

In [59]:
data_37.to_excel('고용표_최종(37).xlsx')

In [60]:
# 각 최종분류의 대분류 수 
five_name_count = [2, 14, 3, 12, 6]

In [61]:
# 각 최종분류의 이름 및 개수 
twenty_name_count = [('농림수산광업', 2), ('전기, 가스 수도사업', 2), ('건설업', 1), ('음식료품제조업', 1), ('코크스 및 석유정제품 제조업', 1), ('화학물질 및 화학제품 제조업', 1), ('공간정보외 기기 및 용품 제조업', 1), ('전기장비 제조업', 1), 
               ('운송장비 제조업', 1), ('제조업', 8), ('운수업', 1), ('숙박 및 음식점업', 1), ('공간정보외 정보통신업', 1), ('금융 및 보험업', 1),
               ('부동산업', 1), ('공간정보외 전문, 과학 및 기술관련 서비스업', 1), ('사업 지원 서비스업', 1), ('의료, 보건업 및 사회복지서비스업', 1), ('서비스업', 4), ('공간정보산업', 6)]
twentyfive_name_count = [('농림수산광업', 2), ('전기, 가스 수도사업', 2), ('건설업', 1), ('음식료품제조업', 1), ('코크스 및 석유정제품 제조업', 1), ('화학물질 및 화학제품 제조업', 1), ('공간정보외 기기 및 용품 제조업', 1), ('전기장비 제조업', 1), 
               ('운송장비 제조업', 1), ('제조업', 8), ('운수업', 1), ('숙박 및 음식점업', 1), ('공간정보외 정보통신업', 1), ('금융 및 보험업', 1),
               ('부동산업', 1), ('공간정보외 전문, 과학 및 기술관련 서비스업', 1), ('사업 지원 서비스업', 1), ('의료, 보건업 및 사회복지서비스업', 1), ('서비스업', 4), ('공간정보 관련 제조업', 1), ('공간정보 관련 도매업', 1), ('공간정보 관련 출판 및 정보서비스업', 1), ('공간정보 관련 기술서비스업', 1), ('공간정보 관련 교육서비스업', 1) 
, ('공간정보 관련 협회 및 단체', 1)]

In [62]:
five_name = ['농림수산광업', '제조업', 'SOC 산업', '서비스업', '공간정보산업']

twenty_name = ['농림수산광업', '전기, 가스 수도사업', '건설업', '음식료품제조업','코크스 및 석유정제품 제조업', '화학물질 및 화학제품 제조업', '공간정보외 기기 및 용품 제조업','전기장비 제조업', 
               '운송장비 제조업', '제조업', '운수업','숙박 및 음식점업', '공간정보외 정보통신업', '금융 및 보험업', 
               '부동산업', '공간정보외 전문, 과학 및 기술관련 서비스업', '사업 지원 서비스업', '의료, 보건업 및 사회복지서비스업', '서비스업', '공간정보산업']

twentyfive_name = ['농림수산광업', '전기, 가스 수도사업', '건설업', '음식료품제조업','코크스 및 석유정제품 제조업', '화학물질 및 화학제품 제조업', '공간정보외 기기 및 용품 제조업','전기장비 제조업', 
               '운송장비 제조업', '제조업', '운수업','숙박 및 음식점업', '공간정보외 정보통신업', '금융 및 보험업', 
               '부동산업', '공간정보외 전문, 과학 및 기술관련 서비스업', '사업 지원 서비스업', '의료, 보건업 및 사회복지서비스업', '서비스업', '공간정보 관련 제조업', '공간정보 관련 도매업', '공간정보 관련 출판 및 정보서비스업', '공간정보 관련 기술서비스업', '공간정보 관련 교육서비스업' 
, '공간정보 관련 협회 및 단체']

In [63]:
new_twenty_list = [['농림수산품', '광산품'], ['전력, 가스 및 증기', '수도, 폐기물처리 및 재활용서비스'], ['건설'], 
                  ['음식료품'], ['석탄 및 석유제품'], ['화학제품'], ['컴퓨터, 전자 및 광학기기'], ['전기장비'], ['운송장비'], 
                  ['섬유 및 가죽제품', '목재 및 종이, 인쇄', '비금속광물제품', '1차 금속제품', '금속가공제품', '기계 및 장비', '기타 제조업 제품', '제조임가공 및 산업용 장비 수리'], 
                  ['운송서비스'], ['음식점 및 숙박서비스'], ['정보통신 및 방송 서비스'], ['금융 및 보험 서비스'], ['부동산서비스'], 
                  ['전문, 과학 및 기술 서비스'], ['사업지원서비스'], ['보건 및 사회복지 서비스'], ['공공행정, 국방 및 사회보장', '예술, 스포츠 및 여가 관련 서비스', '기타 서비스', '기타'], 
                  ['공간정보 관련 제조업', '공간정보 관련 도매업', '공간정보 관련 출판 및 정보서비스업', '공간정보 관련 기술서비스업', 
                  '공간정보 관련 교육서비스업', '공간정보 관련 협회 및 단체']]

new_twentyfive_list = [['농림수산품', '광산품'], ['전력, 가스 및 증기', '수도, 폐기물처리 및 재활용서비스'], ['건설'], 
                  ['음식료품'], ['석탄 및 석유제품'], ['화학제품'], ['컴퓨터, 전자 및 광학기기'], ['전기장비'], ['운송장비'], 
                  ['섬유 및 가죽제품', '목재 및 종이, 인쇄', '비금속광물제품', '1차 금속제품', '금속가공제품', '기계 및 장비', '기타 제조업 제품', '제조임가공 및 산업용 장비 수리'], 
                  ['운송서비스'], ['음식점 및 숙박서비스'], ['정보통신 및 방송 서비스'], ['금융 및 보험 서비스'], ['부동산서비스'], 
                  ['전문, 과학 및 기술 서비스'], ['사업지원서비스'], ['보건 및 사회복지 서비스'], ['공공행정, 국방 및 사회보장', '예술, 스포츠 및 여가 관련 서비스', '기타 서비스', '기타'], 
                  ['공간정보 관련 제조업'], ['공간정보 관련 도매업'], ['공간정보 관련 출판 및 정보서비스업'], ['공간정보 관련 기술서비스업'], 
                  ['공간정보 관련 교육서비스업'], ['공간정보 관련 협회 및 단체']]

In [64]:
# 대분류 최종분류 매칭 사전 생성 (분류표가 이미 정렬되어 있는 경우)
def disting_big(big_name_data, final_name, big_name_count=None):
    i = int(input('총 몇개의 최종분류로 구성되어 있습니까? : '))
    dic = {}
    idx = 0
    yes_no = input('각 최종분류의 포함된 대분류의 수를 이미 작성하신 리스트로 입력하겠습니까? \n 아니면, 직접 입력하겠습니까?(yes or no) ')         
          
    for k in range(i):
        if yes_no == 'yes':
            s = big_name_count[k]
        else:
            s = int(input('최종분류 {}: {}에 포함된 대분류가 몇 개 입니까? '.format(k+1, final_name[k])))
            print()
        for j in range(idx, idx+s):
            idx += 1
            dic[big_name_data[j]] = final_name[k]
        print()
    print('완료')
    return dic

# 재분류 사전 생성 함수(소분류 수준)
def re_disting_big(dic, list_or_count=None, big_name_list=None):
    s = int(input('새로운 분류를 몇 개 추가할 것인가요? '))
    print()
    print()
    yes_or_no = input('이미 튜플 세트로 정리한 분류 이름과 개수 리스트가 있으신가요?(yes or no) ')
    print()
    yes_or_no2 = input('해당 분류에 속할 소분류 이름을 정리한 리스트가 있으신가요?(yes or no) ')        
    
    for i in range(s):
        if yes_or_no == 'yes':
            name = list_or_count[i][0]
            count_d = list_or_count[i][1]
        else:
            print('새로운 분류 이름과 해당 분류에 포함되는 소분류의 개수를 입력해주세요')
            name = input('새로운 분류 이름: ')
            count_d = int(input('{}에 포함될 소분류 개수: '.format(name)))
        print()
        for k in range(count_d):
            if yes_or_no2 == 'yes':
                small_name = big_name_list[i][k]
            else:
                small_name = input('{}에 포함될 {}번 째 소분류명을 입력해주세요 '.format(name, k+1))
            dic[small_name] = name 
        print(name, '분류 생성 완료')
        print()
    print('재분류 완료')
    return dic     

In [66]:
five_dict = disting_big(big_name, five_name, five_name_count)

총 몇개의 최종분류로 구성되어 있습니까? : 5
각 최종분류의 포함된 대분류의 수를 이미 작성하신 리스트로 입력하겠습니까? 
 아니면, 직접 입력하겠습니까?(yes or no) yes





완료


In [67]:
twenty_dict = re_disting_big(five_dict.copy(), twenty_name_count, new_twenty_list)

새로운 분류를 몇 개 추가할 것인가요? 20


이미 튜플 세트로 정리한 분류 이름과 개수 리스트가 있으신가요?(yes or no) yes

해당 분류에 속할 소분류 이름을 정리한 리스트가 있으신가요?(yes or no) yes

농림수산광업 분류 생성 완료


전기, 가스 수도사업 분류 생성 완료


건설업 분류 생성 완료


음식료품제조업 분류 생성 완료


코크스 및 석유정제품 제조업 분류 생성 완료


화학물질 및 화학제품 제조업 분류 생성 완료


공간정보외 기기 및 용품 제조업 분류 생성 완료


전기장비 제조업 분류 생성 완료


운송장비 제조업 분류 생성 완료


제조업 분류 생성 완료


운수업 분류 생성 완료


숙박 및 음식점업 분류 생성 완료


공간정보외 정보통신업 분류 생성 완료


금융 및 보험업 분류 생성 완료


부동산업 분류 생성 완료


공간정보외 전문, 과학 및 기술관련 서비스업 분류 생성 완료


사업 지원 서비스업 분류 생성 완료


의료, 보건업 및 사회복지서비스업 분류 생성 완료


서비스업 분류 생성 완료


공간정보산업 분류 생성 완료

재분류 완료


In [68]:
twenty_five_dict = re_disting_big(five_dict.copy(), twentyfive_name_count, new_twentyfive_list)

새로운 분류를 몇 개 추가할 것인가요? 25


이미 튜플 세트로 정리한 분류 이름과 개수 리스트가 있으신가요?(yes or no) yes

해당 분류에 속할 소분류 이름을 정리한 리스트가 있으신가요?(yes or no) yes

농림수산광업 분류 생성 완료


전기, 가스 수도사업 분류 생성 완료


건설업 분류 생성 완료


음식료품제조업 분류 생성 완료


코크스 및 석유정제품 제조업 분류 생성 완료


화학물질 및 화학제품 제조업 분류 생성 완료


공간정보외 기기 및 용품 제조업 분류 생성 완료


전기장비 제조업 분류 생성 완료


운송장비 제조업 분류 생성 완료


제조업 분류 생성 완료


운수업 분류 생성 완료


숙박 및 음식점업 분류 생성 완료


공간정보외 정보통신업 분류 생성 완료


금융 및 보험업 분류 생성 완료


부동산업 분류 생성 완료


공간정보외 전문, 과학 및 기술관련 서비스업 분류 생성 완료


사업 지원 서비스업 분류 생성 완료


의료, 보건업 및 사회복지서비스업 분류 생성 완료


서비스업 분류 생성 완료


공간정보 관련 제조업 분류 생성 완료


공간정보 관련 도매업 분류 생성 완료


공간정보 관련 출판 및 정보서비스업 분류 생성 완료


공간정보 관련 기술서비스업 분류 생성 완료


공간정보 관련 교육서비스업 분류 생성 완료


공간정보 관련 협회 및 단체 분류 생성 완료

재분류 완료


In [69]:
five_data, five_name = groupping(data_37, five_dict, five_name, 5)

In [70]:
five_data

,취업자수_2015,피용자수_2015,취업자수_2016,피용자수_2016,취업자수_2017,피용자수_2017,취업자수_2018,피용자수_2018,취업자수_2019,피용자수_2019
농림수산광업,1239483.5,122944.3,1180642.2,123713.2,1191843.7,124431.9,1240033.0,122687.5,1289951.0,116319.0
제조업,3850255.9,3397449.1,3832028.1,3367055.2,3802179.5,3331163.9,3742232.6,3301328.1,3650671.1,3242269.9
SOC 산업,1850710.6,1432942.4,1885113.4,1473304.5,1990423.3,1555615.4,2008796.5,1596235.5,1993109.0,1600350.6
서비스업,10852326.1,8317148.9,11069102.2,8499727.2,11199756.1,8551142.4,11220155.0,8598602.9,11342130.4,8729002.1
공간정보산업,6329424.6,4328072.3,6312132.9,4320644.1,6345651.5,4398837.0,6284180.5,4375877.6,6280723.5,4421113.1
계,24122200.7,17598557.0,24279018.8,17784444.2,24529854.1,17961190.6,24495397.6,17994731.6,24556585.0,18109054.7


In [71]:
data_37.drop('대분류', axis=1, inplace=True)

In [72]:
twenty_data, twenty_name = groupping(data_37, twenty_dict, twenty_name, 20)

In [73]:
twenty_data

,취업자수_2015,피용자수_2015,취업자수_2016,피용자수_2016,취업자수_2017,피용자수_2017,취업자수_2018,피용자수_2018,취업자수_2019,피용자수_2019
농림수산광업,1239483.5,122944.3,1180642.2,123713.2,1191843.7,124431.9,1240033.0,122687.5,1289951.0,116319.0
"전기, 가스 수도사업",189654.5,172340.5,197492.8,178430.2,198672.0,175907.5,204832.3,182956.7,209786.4,188100.1
건설업,1661056.1,1260601.9,1687620.6,1294874.3,1791751.3,1379707.9,1803964.2,1413278.8,1783322.6,1412250.5
음식료품제조업,308474.2,237796.6,326307.6,249776.1,339589.1,264638.6,337359.9,262279.0,332904.5,264835.6
코크스 및 석유정제품 제조업,11766.8,11601.3,10165.2,10152.8,11091.7,11077.3,10808.2,10620.5,11197.5,11197.5
화학물질 및 화학제품 제조업,411165.0,378061.0,426457.1,390742.2,440977.9,406055.4,449911.4,417124.9,434660.0,400718.1
공간정보외 기기 및 용품 제조업,261604.3,249651.9,239756.3,228884.0,260356.5,246250.6,256403.3,242813.5,238684.6,227376.6
전기장비 제조업,239244.2,220524.5,235202.0,213460.9,243427.2,219684.8,248625.5,223622.6,238734.6,219090.0
운송장비 제조업,486639.4,473226.1,494411.0,481287.4,491018.9,474427.2,456124.3,442769.4,447436.6,434110.8
제조업,2131362.0,1826587.7,2099728.9,1792751.8,2015718.2,1709030.0,1983000.0,1702098.2,1947053.3,1684941.3


In [74]:
data_37.drop('대분류', axis=1, inplace=True)

In [75]:
twentyfive_data, twentyfive_name = groupping(data_37, twenty_five_dict, twentyfive_name, 25)

In [76]:
twentyfive_data

,취업자수_2015,피용자수_2015,취업자수_2016,피용자수_2016,취업자수_2017,피용자수_2017,취업자수_2018,피용자수_2018,취업자수_2019,피용자수_2019
농림수산광업,1239483.5,122944.3,1180642.2,123713.2,1191843.7,124431.9,1240033.0,122687.5,1289951.0,116319.0
"전기, 가스 수도사업",189654.5,172340.5,197492.8,178430.2,198672.0,175907.5,204832.3,182956.7,209786.4,188100.1
건설업,1661056.1,1260601.9,1687620.6,1294874.3,1791751.3,1379707.9,1803964.2,1413278.8,1783322.6,1412250.5
음식료품제조업,308474.2,237796.6,326307.6,249776.1,339589.1,264638.6,337359.9,262279.0,332904.5,264835.6
코크스 및 석유정제품 제조업,11766.8,11601.3,10165.2,10152.8,11091.7,11077.3,10808.2,10620.5,11197.5,11197.5
화학물질 및 화학제품 제조업,411165.0,378061.0,426457.1,390742.2,440977.9,406055.4,449911.4,417124.9,434660.0,400718.1
공간정보외 기기 및 용품 제조업,261604.3,249651.9,239756.3,228884.0,260356.5,246250.6,256403.3,242813.5,238684.6,227376.6
전기장비 제조업,239244.2,220524.5,235202.0,213460.9,243427.2,219684.8,248625.5,223622.6,238734.6,219090.0
운송장비 제조업,486639.4,473226.1,494411.0,481287.4,491018.9,474427.2,456124.3,442769.4,447436.6,434110.8
제조업,2131362.0,1826587.7,2099728.9,1792751.8,2015718.2,1709030.0,1983000.0,1702098.2,1947053.3,1684941.3


In [77]:
five_data.to_excel('고용표_최종(5).xlsx')
twenty_data.to_excel('고용표_최종(20).xlsx')
twentyfive_data.to_excel('고용표_최종(25).xlsx')